### A little 'get started with the LIMS / PostgreSQL / Python connection' notebook following up on the Software Carpentry Workshop.
####  Agata Budzillo, 4/12/2017

### The three functions below will allow you to open a connection with the LIMS database, send a query and collect the results, and then (importantly) close the connection so LIMS doesn't get hung up. I wouldn't worry too much about the details of these at the moment, but I'm happy to answer questions if they arise.

In [1]:

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    import pg8000
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        cursor.close()
        conn.close()
    return results


### LIMS has a looooot of tables. One of the most useful ones to get acquainted with is 'specimens'.

In [2]:
my_bigquery = "SELECT * FROM specimens"
my_bigresult = limsquery(my_bigquery)

### Well now that took an awfully long time. From now on when we run our test queries, let's limit the number of results we get back.

In [3]:
my_query = "SELECT * FROM specimens LIMIT 10"
my_result = limsquery(my_query)

### What in the world is in my result? Let's ask Python what it thinks...

In [173]:
print type(my_result)
print "Length of myresult:", len(my_result)

<type 'list'>
Length of myresult: 10


### It's a list of the same length as the # of results to which we limited our query.  What's the first thing in the list?

In [174]:
first_element = myresult[0]
print first_element

{'cell_depth': None, 'ephys_roi_result_id': None, 'parent_y_coord': 0, 'reference_space_id': None, 'updated_at': datetime.datetime(2016, 12, 16, 4, 54, 44, 477335), 'cell_label': None, 'preparation_method_id': None, 'parent_x_coord': 2, 'location_id': None, 'id': 556516441, 'cortex_layer_id': None, 'plane_of_section_id': 11, 'frozen_at': None, 'flipped_specimen_id': 561557765, 'data': None, 'rna_integrity_number': None, 'histology_well_name': None, 'created_by': None, 'priority': None, 'parent_id': 556516212, 'project_id': 305094322, 'alignment3d_id': None, 'carousel_well_name': u'T301_122_161107_01_12', 'patched_cell_container': None, 'updated_by': None, 'cell_prep_id': None, 'biophysical_model_state': u'review_required', 'barcode': u'0556516441', 'storage_directory': None, 'tissue_ph': None, 'specimen_preparation_method_id': None, 'donor_id': 555257198, 'ephys_neural_tissue_plan_id': 555257244, 'structure_id': None, 'parent_z_coord': 0, 'facs_well_id': None, 'name': u'Ndnf-IRES2-dgCr

### Eew, what is that!?

In [176]:
print type(first_element)

<type 'dict'>


### Aha, Python says it's a dictionary! This is an incredibly useful and efficient data structure that we didn't really discuss in the Software Carpentry Workshop. A dictionary is made up of key:value pairs. Just to make that clear, below I'm looping across every key in the dictionary that is the first element of my results list, printing the name of the key and the value that the key is associated with.

In [178]:
my_dict = first_element
for my_key in my_dict.keys():
    my_val = my_dict[my_key]
    print my_key, ":", my_val

cell_depth : None
ephys_roi_result_id : None
parent_y_coord : 0
reference_space_id : None
updated_at : 2016-12-16 04:54:44.477335
cell_label : None
preparation_method_id : None
parent_x_coord : 2
location_id : None
id : 556516441
cortex_layer_id : None
plane_of_section_id : 11
frozen_at : None
flipped_specimen_id : 561557765
data : None
rna_integrity_number : None
histology_well_name : None
created_by : None
priority : None
parent_id : 556516212
project_id : 305094322
alignment3d_id : None
carousel_well_name : T301_122_161107_01_12
patched_cell_container : None
updated_by : None
cell_prep_id : None
biophysical_model_state : review_required
barcode : 0556516441
storage_directory : None
tissue_ph : None
specimen_preparation_method_id : None
donor_id : 555257198
ephys_neural_tissue_plan_id : 555257244
structure_id : None
parent_z_coord : 0
facs_well_id : None
name : Ndnf-IRES2-dgCre;Ai14-280612.06.02
normalization_group_id : None
postmortem_interval_id : None
specimen_set_id : None
create

## Q1: Your turn - return the specimen names for all the query results.

In [179]:
# A1:
for result in my_result:
    print result["name"]


Ndnf-IRES2-dgCre;Ai14-280612.06.02
Ai14-Homo-280711.10
Sst-IRES-Cre;Ai140;Pvalb-2A-FlpO;Ai65F-303812
Htr3a-Cre_NO152;Ai14-288785
H16.03.002.01.06.02
Slc17a6-IRES-Cre-125964
Sim1-Cre_KJ18;Ai14-264358.16
Sim1-Cre_KJ18;Ai14-264358.10
Cux2-CreERT2-124835
Sim1-Cre_KJ18;Ai14-264358.11


### Dictionaries are cool, but since this is tabular data, it might make sense to load it into a Pandas Dataframe, so it's all nice and pretty. Here's one way of doing it.

In [181]:
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    import pandas as pd
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

In [184]:
my_df = get_lims_dataframe(myquery)
my_df.head()

,cell_depth,ephys_roi_result_id,parent_y_coord,reference_space_id,updated_at,cell_label,preparation_method_id,parent_x_coord,location_id,id,...,normalization_group_id,postmortem_interval_id,specimen_set_id,created_at,tissue_processing_id,ephys_cell_plan_id,hemisphere_id,cell_reporter_id,task_flow_id,external_specimen_name
0,NaN,NaN,0.0,None,2016-12-16 04:54:44.477335,None,None,2.0,None,556516441,...,None,None,None,2016-11-07 16:03:11.459275,555257241.0,NaN,None,None,None,None
1,NaN,NaN,0.0,None,2016-12-16 04:54:44.502455,None,None,0.0,None,557340893,...,None,None,None,2016-11-10 18:44:08.892340,553464045.0,NaN,None,None,None,None
2,NaN,NaN,NaN,None,2017-03-01 16:37:56.349162,None,None,NaN,None,571099320,...,None,None,None,2017-02-24 16:19:14.340373,571099382.0,NaN,None,None,None,None
3,NaN,NaN,NaN,None,2016-12-20 15:59:41.497432,None,None,NaN,None,561463630,...,None,None,None,2016-12-14 17:16:20.787949,561463640.0,NaN,None,None,None,None
4,79.5,520465696.0,0.0,None,2016-12-16 04:20:18.220136,None,None,0.0,None,520465880,...,None,None,None,2016-05-25 01:53:53.036673,NaN,497017362.0,None,None,None,None


### OK  well I don't know about you, but I'm annoyed by all these useless columns here. Who cares about tissue_processing_id!? Just kidding... We can limit what fields we take back from LIMS. I showed you the first way (with the wildcard *), though, because sometimes it can be tricky to know what the available fieldnames in a given table are ahead of time. For example, they often do not match up to the names displayed in the LIMS tables you may be accustomed to exploring through the web interface. Let's say we just want the specimen name, the ephys roi id and the specimen id....

In [187]:
my_refinedquery = "SELECT name, id, ephys_roi_result_id FROM specimens LIMIT 5"
refined_df = get_lims_dataframe(my_refinedquery)
myrefined_dataframe.tail()

,ephys_roi_result_id,name,id
0,NaN,Ndnf-IRES2-dgCre;Ai14-280612.06.02,556516441
1,NaN,Ai14-Homo-280711.10,557340893
2,NaN,Sst-IRES-Cre;Ai140;Pvalb-2A-FlpO;Ai65F-303812,571099320
3,NaN,Htr3a-Cre_NO152;Ai14-288785,561463630
4,520465696.0,H16.03.002.01.06.02,520465880


### Sweet. Now, are these all electrophysiology specimens? Why do some of them have odd names and no roi id? It's because specimens in LIMS are all specimens, not just electrophysiology specimens. So how do we narrow down to ephys specimens? That will require a table merge. It's kind of a weird one, though as we will have to merge the table with itself.  Whhaaaat? Yeah. So before we do that, let's work on a more 'traditional' merge.  Let's work with a couple of new tables, ephys_roi_results and users to find out which recordings a given rig user served as stage 1 reviewer for..." 

In [195]:
new_query = "SELECT id, recording_date, stage1_reviewer_id, workflow_state FROM ephys_roi_results"
err_df = get_lims_dataframe(new_query)
err_df.head()

,workflow_state,id,recording_date,stage1_reviewer_id
0,auto_failed,539646734,2016-08-17 10:18:51,NaN
1,qc,306486658,NaT,NaN
2,qc,305384219,NaT,NaN
3,auto_failed,539011260,2016-08-12 11:04:58,NaN
4,qc,306007448,NaT,NaN


### Hmm seeing a lot of NaNs in stage1_reviewer_id? Remember that we only implemented this into LIMS this past fall. Can you figure out how to limit the results to only 2017? The syntax of the wrapper for PostGreSQL in Python is a little funny. Hint: the date may need to be passed in quotes.

In [196]:
new_query = "SELECT id, recording_date, stage1_reviewer_id, workflow_state \
FROM ephys_roi_results WHERE recording_date > '2017-01-01'"
err_df = get_lims_dataframe(new_query)
err_df.head()

,workflow_state,id,recording_date,stage1_reviewer_id
0,auto_failed,569387813,2017-02-08 10:52:32,NaN
1,auto_failed,580133753,2017-04-07 10:32:16,NaN
2,manual_passed,570428511,2017-02-21 09:44:42,305127608.0
3,manual_passed,571255031,2017-02-27 11:12:23,525757437.0
4,auto_failed,575894497,2017-03-22 09:30:05,NaN


### Nice. There are still NaNs, but only where the workflow state was dodgy. So there is another table that could come in handy now, called users. What's in this table? 

In [198]:
users_query = "SELECT id, login FROM users"
users_df = get_lims_dataframe(users_query)

### Let's find your user id given your Allen login.

In [200]:
users_df = get_lims_dataframe("SELECT id, login FROM users WHERE login = 'agatab'")
users_df.head()

,login,id
0,agatab,527452252


### Sweet! So all we need to do now is JOIN the users table on our ephys results table to figure out which samples you were a stage 1 reviewer for... JOINs can be a little complicated, so I recommend you tread carefully. Come ask if you need help or if your results do not make sense. The syntax is the following. You'll want to list all of the field names from all of the tables you want at the beginning in the SELECT statement, then your FROM statement and then any number of JOIN (table name) ON (column name). The filters (ie WHERE, LIMIT, ORDER BY) always go at the end. By default JOIN will be an INNER JOIN (think the overlapping section in a Venn diagram). This is also a good time to bring up aliasing. Aliasing is IMO always a good idea. Imagine you write the following: 

In [201]:
join_query = "SELECT id, recording_date, stage1_reviewer_id, workflow_state, id, login \
FROM ephys_roi_results \
JOIN users ON stage1_reviewer_id = id \
WHERE recording_date > '2017-01-01'"
join_df = get_lims_dataframe(join_query)

ProgrammingError: (u'ERROR', u'42702', u'column reference "id" is ambiguous', u'132', u'parse_relation.c', u'654', u'colNameToVar', u'', u'')

###  Well, that was an epic fail. Both ephys_roi_results and users have an id column so Python is not happy. In fact if you look at the bottom of the traceback the error message is : 'column reference "id" is ambiguous'. The proper way to do this is to explicitly call the field name from the table, like so:


In [203]:
join_query = "SELECT ephys_roi_results.id, ephys_roi_results.recording_date, ephys_roi_results.stage1_reviewer_id, \
ephys_roi_results.workflow_state, users.id, users.login \
FROM ephys_roi_results \
JOIN users ON ephys_roi_results.stage1_reviewer_id = users.id \
WHERE recording_date > '2017-01-01'"
join_df = get_lims_dataframe(join_query)
join_df.tail()

,login,workflow_state,id,recording_date,stage1_reviewer_id
705,samj,manual_passed,525757437,2017-01-06 09:07:31,525757437
706,samj,manual_failed,525757437,2017-01-04 15:42:32,525757437
707,samj,manual_passed,525757437,2017-01-05 14:39:22,525757437
708,samj,manual_passed,525757437,2017-01-05 13:28:19,525757437
709,samj,manual_passed,525757437,2017-01-04 10:17:49,525757437


### That works! But it's super clunky. That's where aliases comes in handy. "'Alias' was a show about a spy...."

In [209]:
join_query = "SELECT err.id, err.recording_date, err.stage1_reviewer_id, \
err.workflow_state, users.id, users.login \
FROM ephys_roi_results err \
JOIN users ON err.stage1_reviewer_id = users.id \
WHERE recording_date > '2017-01-01'"
join_df = get_lims_dataframe(join_query)
join_df.tail()

,login,workflow_state,id,recording_date,stage1_reviewer_id
705,samj,manual_passed,525757437,2017-01-06 09:07:31,525757437
706,samj,manual_failed,525757437,2017-01-04 15:42:32,525757437
707,samj,manual_passed,525757437,2017-01-05 14:39:22,525757437
708,samj,manual_passed,525757437,2017-01-05 13:28:19,525757437
709,samj,manual_passed,525757437,2017-01-04 10:17:49,525757437


### You can also alias the columns:

In [210]:
join_query = "SELECT err.id AS roi_id, err.recording_date AS date, err.stage1_reviewer_id AS st1r, \
err.workflow_state AS qcstate, users.id, users.login AS username \
FROM ephys_roi_results err \
JOIN users ON err.stage1_reviewer_id = users.id \
WHERE recording_date > '2017-01-01'"
join_df = get_lims_dataframe(join_query)
join_df.tail()

,username,qcstate,roi_id,st1r,date,id
705,samj,manual_passed,562630185,525757437,2017-01-06 09:07:31,525757437
706,samj,manual_failed,562438408,525757437,2017-01-04 15:42:32,525757437
707,samj,manual_passed,562540356,525757437,2017-01-05 14:39:22,525757437
708,samj,manual_passed,562534360,525757437,2017-01-05 13:28:19,525757437
709,samj,manual_passed,562381380,525757437,2017-01-04 10:17:49,525757437


### OK, this is great, but those roi_ids are so unsatisfying. Wouldn't it be awesome if we could actually recover the names of those specimens? Let's jump back to our original task.The weird thing here is that we're joining one column of the specimens table on another column of the specimens table. I've aliased each 'instance' of the specimens table as cell or slice, so hopefully this makes sense. Finally we can join the result on the ephys roi results table.

In [212]:
cell_query = "SELECT cell.id AS cell_id, cell.name AS cell_name, err.id AS roi_id, err.recording_date AS date, \
err.workflow_state as qcstate \
FROM specimens cell \
JOIN specimens slice ON cell.parent_id = slice.id \
JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id \
WHERE err.recording_date > '2015-01-01'"
cell_df = get_lims_dataframe(cell_query)
cell_df.head()


,date,qcstate,cell_id,roi_id,cell_name
0,2015-01-05 11:07:44,manual_passed,318331264,318331262,Pvalb-IRES-Cre;Ai14-169125.02.02.01
1,2015-01-05 11:12:00,manual_passed,318331342,318331340,Pvalb-IRES-Cre;Ai14-169125.03.01.01
2,2015-01-05 11:33:20,auto_passed,318331484,318331482,Pvalb-IRES-Cre;Ai14-169125.03.02.01
3,2015-01-05 13:15:44,auto_failed,318346915,318346913,Pvalb-IRES-Cre;Ai14-169125.04.02.01
4,2015-01-05 13:49:52,auto_passed,318354031,318354029,Pvalb-IRES-Cre;Ai14-169128.03.02.01


### Question: Now can you put all this together, and figure out which cell names you were a stage 1 reviewer for...?

### That concludes this little notebook. I hope it was useful. If you want to learn/practice more SQL syntax without the context of the Python wrapper for our PostgreSQL LIMS database, I recommend the following resources:
https://www.w3schools.com/sql/
https://community.modeanalytics.com/sql/tutorial/introduction-to-sql/